# Safety Equipment Video Moderation with Pydantic AI

This notebook demonstrates how to build a video moderation system that enforces workplace safety policies using Pydantic AI and Gemini. Our system will detect whether employees are wearing required safety equipment and flag violations with timestamps.

## Import Libraries and Load Environment

Make sure you have your Google Gemini credentials in the `.env` file:

```bash
cp env.example .env
```
then edit `.env` and modify GEMINI_API_KEY with your key.

In [ ]:
import os
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from pydantic_ai import Agent, BinaryContent
from IPython.display import Video, display
from typing import List, Optional

# Load environment variables
assert load_dotenv(), "Please prepare a .env file with your GEMINI_API_KEY"
assert os.getenv("GEMINI_API_KEY"), "GEMINI_API_KEY not found in .env file"

# Only needed on the Udacity workspace. Comment this out if running on another system.
os.environ['HF_HOME'] = '/voc/data/huggingface'
os.environ['OLLAMA_MODELS'] = '/voc/data/ollama/cache'
os.environ['HF_HUB_OFFLINE'] = '1'
os.environ['PATH'] = f"/voc/data/ollama/bin:/voc/data/ffmpeg/bin:{os.environ.get('PATH', '')}"
os.environ['LD_LIBRARY_PATH'] = f"/voc/data/ollama/lib:/voc/data/ffmpeg/lib:{os.environ.get('LD_LIBRARY_PATH', '')}"

# This is needed to use asyncio within jupyter
import nest_asyncio
nest_asyncio.apply()

## Define Safety Policy Schema

We'll use Pydantic models to structure our safety compliance analysis. This ensures consistent detection of policy violations that can be processed automatically.

In [21]:
from typing_extensions import Literal

class SafetyViolation(BaseModel):
    """Individual safety violation detected in video."""
    
    violation_type: Literal["missing_hard_hat", "missing_mask"] = Field(
        description="Type of safety equipment violation detected"
    )
    
    timestamp_start: float = Field(
        description="Start time in seconds when violation begins"
    )
    
    timestamp_end: float = Field(
        description="End time in seconds when violation ends"
    )

class SafetyModerationResult(BaseModel):
    """Complete safety moderation analysis for a video."""
    
    overall_compliance: Literal["compliant", "violations_detected"] = Field(
        description="Overall safety compliance status"
    )
    
    violations: List[SafetyViolation] = Field(
        description="List of all safety violations detected"
    )
    
    summary: str = Field(
        description="Brief summary of safety compliance findings"
    )

## Create Safety Moderation Agent

Set up the Pydantic AI agent with specific instructions for detecting safety equipment violations.

In [23]:
from pydantic_ai.models.google import GoogleModelSettings

# Remove thinking to avoid long delays
settings = GoogleModelSettings(
    google_thinking_config={"thinking_budget": 0},
    seed=42,
    temperature=0.0
)

safety_agent = Agent(
    model="gemini-2.5-flash-lite",
    output_type=SafetyModerationResult,
    instructions="""
    You are a workplace safety compliance system. Analyze the provided video to detect 
    violations of safety equipment policy.
    
    REQUIRED SAFETY EQUIPMENT:
    - Hard hats (helmets) must be worn by all personnel
    - Masks must be worn by all personnel
    
    Your tasks:
    1. Identify all people visible in the video
    2. Check if each person is wearing required safety equipment
    3. For every violation, fill out a SafetyViolation entry with:
        - Type of violation (missing hard hat, missing mask)
        - Timestamps for when the violation starts and ends (start and stop)
       Note that the same person could have multiple violations over time.
        
    4. Determine overall compliance:
        - "compliant" if no violations are found
        - "violations_detected" if any violations are found
    5. Provide a brief summary of your findings
    
    Be precise with timestamps and focus only on clearly visible safety equipment violations.
    """,
    model_settings=settings,
    retries=3,
)

## Helper Functions

Define functions to load videos and handle safety violation alerts.

In [24]:
def load_video_for_analysis(video_path):
    """Load video file and format it for Pydantic AI."""
    with open(video_path, 'rb') as f:
        video_bytes = f.read()
    
    video_content = BinaryContent(
        data=video_bytes,
        media_type='video/mp4'
    )
    
    return video_content

def alert_safety_official(violation: str):
    """Fake function to simulate alerting safety officials about violations."""
    print(f"🚨 SAFETY ALERT SENT: {violation}")
    print(f"   → Notification sent to safety@company.com")
    print(f"   → Incident logged in safety management system")
    print(f"   → Supervisor notified for immediate action\n")

## Safety Video Analysis

Analyze a workplace video to detect safety equipment violations and generate compliance reports.

In [25]:
# Replace with your video file path
from pprint import pprint

for video in ["construction1.mp4", "construction2.mp4"]:
    video_path = video  # Change this to your video file

    # Display the video
    print("Analyzing workplace safety video:")
    display(Video(video_path))

    # Load and analyze the video
    video_content = load_video_for_analysis(video_path)

    print("\nRunning safety compliance analysis...")
    result = safety_agent.run_sync(
        ["Analyze this video for workplace safety equipment compliance.", video_content]
    )

    safety_result = result.output
    pprint(safety_result.model_dump())

    print("".join(["-"*40]*2))

Analyzing workplace safety video:



Running safety compliance analysis...
{'overall_compliance': 'compliant',
 'summary': 'All personnel were observed to be wearing the required hard hats '
            'and masks throughout the video.',
 'violations': []}
--------------------------------------------------------------------------------
Analyzing workplace safety video:



Running safety compliance analysis...
{'overall_compliance': 'violations_detected',
 'summary': 'The individual in the video is not wearing a hard hat or a mask '
            'during the demolition work, which is a violation of safety '
            'policies.',
 'violations': [{'timestamp_end': 19.0,
                 'timestamp_start': 0.0,
                 'violation_type': 'missing_hard_hat'},
                {'timestamp_end': 19.0,
                 'timestamp_start': 0.0,
                 'violation_type': 'missing_mask'}]}
--------------------------------------------------------------------------------


## Process Violations and Send Alerts

When violations are detected, process each one and notify safety officials with specific details.

In [18]:
if safety_result.violations:
    print(f"\n⚠️  {len(safety_result.violations)} SAFETY VIOLATION(S) DETECTED\n")
    
    for i, violation in enumerate(safety_result.violations, 1):
        print(f"Violation #{i}:")
        print(f"  Type: {violation.violation_type.replace('_', ' ').title()}")
        print(f"  Time: {violation.timestamp_start}s - {violation.timestamp_end}s")
        
        # Generate alert message
        alert_message = (
            f"{violation.violation_type.replace('_', ' ').title()} detected "
            f"from {violation.timestamp_start}s to {violation.timestamp_end}s. "
        )
        
        # Send alert to safety officials
        alert_safety_official(alert_message)
else:
    print("\n✅ NO SAFETY VIOLATIONS DETECTED")
    print("All personnel observed were wearing required safety equipment.")


⚠️  2 SAFETY VIOLATION(S) DETECTED

Violation #1:
  Type: Missing Hard Hat
  Time: 0.0s - 19.0s
🚨 SAFETY ALERT SENT: Missing Hard Hat detected from 0.0s to 19.0s. 
   → Notification sent to safety@company.com
   → Incident logged in safety management system
   → Supervisor notified for immediate action

Violation #2:
  Type: Missing Safety Vest
  Time: 0.0s - 19.0s
🚨 SAFETY ALERT SENT: Missing Safety Vest detected from 0.0s to 19.0s. 
   → Notification sent to safety@company.com
   → Incident logged in safety management system
   → Supervisor notified for immediate action

